In [12]:
#Importamos la librería pandas que necesitamos para la lectura, conversión y limpieza de los datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
import re 

#Importamos librerías necesarias para la visualización
# -----------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

#Importamos librerías para localizar ruta archivos.
import os
import sys
sys.path.append("../")

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
pd.set_option('display.max_rows', None) # para poder visualizar todas las filas de los DataFrames

In [13]:
# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
import mysql.connector
from mysql.connector import errorcode



In [14]:
df_HR = pd.read_csv('/Users/Tania_1/Desktop/ADALAB/MODULO_3/project-da-promo-angela-modulo-3-team-3/ETL/data/output_data/df_HR_limpio.csv')
df_HR.head()

,Age,Attrition,BusinessTravel,DailyRate,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,DateBirth,RemoteWork
0,51,No,Unknown,684.0,6,3,Unknown,"162,0",1,M,51.0,3,5,Research Director,3,Unknown,19537.0,6462,7,No,13,"3,0",3,0,Unknown,5,3,20,15,15,1972,Yes
1,52,No,Unknown,699.0,1,4,Life Sciences,"259,0",3,M,65.0,2,5,Manager,3,Unknown,19999.0,5678,0,Unknown,14,"3,0",1,1,"34,0",5,3,33,11,9,1971,1
2,42,No,travel_rarely,532.0,4,2,Technical Degree,"319,0",3,M,58.0,3,5,Manager,4,Married,19232.0,4933,1,No,11,"3,0",4,0,"22,0",3,3,22,11,15,1981,1
3,47,No,travel_rarely,359.0,2,4,Medical,Unknown,1,F,82.0,3,4,Research Director,3,Married,17169.0,26703,3,Unknown,19,"3,0",2,2,Unknown,2,3,20,5,6,1976,False
4,48,No,Unknown,117.0,22,3,Medical,"1900,0",4,F,58.0,3,4,Manager,4,Unknown,17174.0,2437,3,No,11,"3,0",2,1,Unknown,3,3,22,4,7,1975,Yes


In [15]:
df_HR.shape

(1080, 32)

In [16]:
df_empleados = pd.read_csv('/Users/Tania_1/Desktop/ADALAB/MODULO_3/project-da-promo-angela-modulo-3-team-3/ETL/data/output_data/Employees.csv')
df_empleados.shape

(1080, 9)

### Crear columna índice en los csvs

In [25]:
#empleados

df_empleados_reset = df_empleados.reset_index()

df_empleados_reset.head()

,index,Age,Attrition,BusinessTravel,DistanceFromHome,Education,EducationField,Gender,MaritalStatus,DateBirth
0,0,51,No,Unknown,6,3,Unknown,M,Unknown,1972
1,1,52,No,Unknown,1,4,Life Sciences,M,Unknown,1971
2,2,42,No,travel_rarely,4,2,Technical Degree,M,Married,1981
3,3,47,No,travel_rarely,2,4,Medical,F,Married,1976
4,4,48,No,Unknown,22,3,Medical,F,Unknown,1975


In [18]:
df_empleados_reset.rename(columns={'index': 'ID_FK'}, inplace=True)


In [19]:
df_empleados_reset.to_csv('Employees.csv')

In [20]:
df = pd.read_csv('Employees.csv', index_col= 0)
df.head(5)

,ID_FK,Age,Attrition,BusinessTravel,DistanceFromHome,Education,EducationField,Gender,MaritalStatus,DateBirth
0,0,51,No,Unknown,6,3,Unknown,M,Unknown,1972
1,1,52,No,Unknown,1,4,Life Sciences,M,Unknown,1971
2,2,42,No,travel_rarely,4,2,Technical Degree,M,Married,1981
3,3,47,No,travel_rarely,2,4,Medical,F,Married,1976
4,4,48,No,Unknown,22,3,Medical,F,Unknown,1975


In [22]:
#Job details

df_job = pd.read_csv('/Users/Tania_1/Desktop/ADALAB/MODULO_3/project-da-promo-angela-modulo-3-team-3/ETL/data/output_data/JobDetails.csv')
df_job.head(5)


,JobLevel,JobRole,NumCompaniesWorked,TotalWorkingYears,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,5,Research Director,7,Unknown,20,15,15
1,5,Manager,0,"34,0",33,11,9
2,5,Manager,1,"22,0",22,11,15
3,4,Research Director,3,Unknown,20,5,6
4,4,Manager,3,Unknown,22,4,7


In [ ]:
df_job_reset = df_job.reset_index()

df_job_reset.sample(5)

,index,JobLevel,JobRole,NumCompaniesWorked,TotalWorkingYears,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
802,802,1,Research Scientist,3,"6,0",3,0,2
796,796,2,Research Scientist,9,Unknown,2,2,1
736,736,3,Healthcare Representative,4,Unknown,3,1,2
463,463,2,Sales Executive,0,Unknown,8,7,1
746,746,2,Healthcare Representative,7,"9,0",6,1,3


In [ ]:
df_job_reset.rename(columns={'index': 'ID_FK'}, inplace=True)
df_job_reset.tail(5)

,ID_FK,JobLevel,JobRole,NumCompaniesWorked,TotalWorkingYears,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
1075,1075,2,Manufacturing Director,8,Unknown,1,0,0
1076,1076,1,Laboratory Technician,1,Unknown,1,0,0
1077,1077,1,Laboratory Technician,8,"8,0",1,0,0
1078,1078,1,Sales Representative,1,Unknown,1,1,0
1079,1079,3,Human Resources,1,Unknown,9,1,7


In [ ]:
df_job_reset.to_csv('JobDetails.csv')

In [23]:
#compensation

df_compensacion = pd.read_csv('/Users/Tania_1/Desktop/ADALAB/MODULO_3/project-da-promo-angela-modulo-3-team-3/ETL/data/output_data/Compensation.csv')
df_compensacion.head(5)

,DailyRate,HourlyRate,MonthlyIncome,MonthlyRate,OverTime,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,RemoteWork
0,684.0,51.0,19537.0,6462,No,13,0,5,Yes
1,699.0,65.0,19999.0,5678,Unknown,14,1,5,1
2,532.0,58.0,19232.0,4933,No,11,0,3,1
3,359.0,82.0,17169.0,26703,Unknown,19,2,2,False
4,117.0,58.0,17174.0,2437,No,11,1,3,Yes


In [ ]:
df_compensacion_reset = df_compensacion.reset_index()

df_compensacion_reset.head()

,index,DailyRate,HourlyRate,MonthlyIncome,MonthlyRate,OverTime,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,RemoteWork
0,0,684.0,51.0,19537.0,6462,No,13,0,5,yes
1,1,699.0,65.0,19999.0,5678,Unknown,14,1,5,yes
2,2,532.0,58.0,19232.0,4933,No,11,0,3,yes
3,3,359.0,82.0,17169.0,26703,Unknown,19,2,2,no
4,4,117.0,58.0,17174.0,2437,No,11,1,3,yes


In [ ]:
df_compensacion_reset.rename(columns={'index': 'ID_FK'}, inplace=True)
df_compensacion_reset.tail(5)

,ID_FK,DailyRate,HourlyRate,MonthlyIncome,MonthlyRate,OverTime,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,RemoteWork
1075,1075,1373.0,56.0,6463.923077,10138,No,24,0,1,yes
1076,1076,1276.0,59.0,1129.000000,17536,Unknown,11,3,4,yes
1077,1077,543.0,83.0,6463.923077,4060,No,19,2,3,yes
1078,1078,638.0,33.0,6463.923077,26956,No,19,1,3,no
1079,1079,1146.0,31.0,6463.923077,16642,Yes,16,0,2,no


In [ ]:
df_compensacion_reset.to_csv('Compensation.csv')

In [24]:
#satisfaccion

df_satisfaccion = pd.read_csv('/Users/Tania_1/Desktop/ADALAB/MODULO_3/project-da-promo-angela-modulo-3-team-3/ETL/data/output_data/Satisfaction.csv')
df_satisfaccion.head(5)

,EnvironmentSatisfaction,JobInvolvement,JobSatisfaction,PerformanceRating,RelationshipSatisfaction,WorkLifeBalance
0,1,3,3,"3,0",3,3
1,3,2,3,"3,0",1,3
2,3,3,4,"3,0",4,3
3,1,3,3,"3,0",2,3
4,4,3,4,"3,0",2,3


In [ ]:
df_satisfaccion_reset = df_satisfaccion.reset_index()

df_satisfaccion_reset.head()

,index,EnvironmentSatisfaction,JobInvolvement,JobSatisfaction,PerformanceRating,RelationshipSatisfaction,WorkLifeBalance
0,0,1,3,3,"3,0",3,3.000000
1,1,3,2,3,"3,0",1,3.000000
2,2,3,3,4,"3,0",4,2.766568
3,3,1,3,3,"3,0",2,2.766568
4,4,4,3,4,"3,0",2,3.000000


In [ ]:
df_satisfaccion_reset.rename(columns={'index': 'ID_FK'}, inplace=True)
df_satisfaccion_reset.tail(5)

,ID_FK,EnvironmentSatisfaction,JobInvolvement,JobSatisfaction,PerformanceRating,RelationshipSatisfaction,WorkLifeBalance
1075,1075,4,2,4,"4,0",2,3.000000
1076,1076,4,1,4,"3,0",3,3.000000
1077,1077,1,3,4,"3,0",3,2.000000
1078,1078,4,3,1,"3,0",1,2.766568
1079,1079,3,3,4,"3,0",3,2.766568


In [ ]:
df_satisfaccion.to_csv('Satisfaction.csv')

Crear base de datos

In [ ]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1')


mycursor = cnx.cursor()
try:
    mycursor.execute("CREATE DATABASE ABC_Corporation")
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: CREATE DATABASE ABC_Corporation


Creacion de tablas

Crear tabla Employees

In [ ]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='ABC_Corporation')


mycursor = cnx.cursor()
try:
    mycursor.execute("""
        CREATE TABLE Employees (
        ID_FK INT PRIMARY KEY,
        Age INT,
        Gender VARCHAR(100),
        MaritalStatus VARCHAR(100),
        DateBirth INT,
        Education INT,
        EducationField VARCHAR(100),
        Attrition VARCHAR(100),
        BusinessTravel VARCHAR(100),
        DistanceFromHome INT
    )
    """)
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: 
        CREATE TABLE Employees (
      ..


### Crear tabla JobDetails

In [ ]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='ABC_Corporation')


mycursor = cnx.cursor()
try:
    mycursor.execute("""
        CREATE TABLE JobDetails (
        ID_FK INT PRIMARY KEY,
        JobRole VARCHAR(100),
        JobLevel INT,
        NumCompaniesWorked INT,
        TotalWorkingYears VARCHAR(100),
        YearsAtCompany INT,
        YearsSinceLastPromotion INT,
        YearsWithCurrManager INT
    )
    """)
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: 
        CREATE TABLE JobDetails (
     ..


### Crear tabla Compensation

In [ ]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='ABC_Corporation')


mycursor = cnx.cursor()
try:
    mycursor.execute("""
        CREATE TABLE Compensation (
        ID_FK INT PRIMARY KEY,
        DailyRate INT,
        HourlyRate INT,
        MonthlyIncome INT,
        MonthlyRate INT,
        StockOptionLevel INT,
        PercentSalaryHike INT,
        OverTime VARCHAR(100),
        TrainingTimesLastYear INT,
        RemoteWork VARCHAR(100)
    )
    """)
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: 
        CREATE TABLE Compensation (
   ..


### Creacion tabla Satisfaction

In [ ]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='ABC_Corporation')


mycursor = cnx.cursor()
try:
    mycursor.execute("""
        CREATE TABLE Satisfaction (
        ID_FK INT PRIMARY KEY,
        EnvironmentSatisfaction INT,
        JobInvolvement INT,
        JobSatisfaction INT,
        RelationshipSatisfaction INT,
        WorkLifeBalance INT,
        PerformanceRating VARCHAR(100)
    )
    """)
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: 
        CREATE TABLE Satisfaction (
   ..
